## 번역 성능 평가 

Kor - Eng

In [1]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
from bert_score import score

In [2]:
# 평가 함수
def evaluate(
    reference: str,
    candidate: str,
    tgt_lang: str,
    bert_model="microsoft/deberta-xlarge-mnli",
):
    # BLEU 점수 계산 (스무딩 적용)
    bleu = sentence_bleu(
        [reference.split()],
        candidate.split(),
        smoothing_function=SmoothingFunction().method1,
    )

    # METEOR 점수 계산
    meteor = meteor_score([reference], candidate)

    # ROUGE 점수 계산
    scorer = rouge_scorer.RougeScorer(["rouge1", "rougeL"], use_stemmer=True)
    scores = scorer.score(reference, candidate)

    # BERTScore 계산
    P, R, F1 = score(
        [candidate],
        [reference],
        lang=tgt_lang,
        model_type=bert_model,
        device="cuda",  # GPU 사용
    )

    return {
        "BLEU": round(bleu, 4),
        "METEOR": round(meteor, 4),
        "ROUGE": round(
            np.mean(np.mean([score.precision for score in scores.values()])), 4
        ),
        "BERT": round(F1.item(), 4),
    }

데이터셋 FLORES-Plus 준비

In [8]:
# Parquet 데이터셋 로드 함수
def load_parquet_data(eng_path, kor_path):
    # 영어-한국어 데이터셋 로드
    eng_df = pd.read_parquet(eng_path)
    kor_df = pd.read_parquet(kor_path)

    # 데이터프레임 합치기 (인덱스 기준으로 병합)
    dataset = pd.concat([eng_df, kor_df], axis=1)
    dataset.columns = ["source", "target"]  # 컬럼 이름 설정
    return dataset

In [10]:
import pyarrow.parquet as pq

# Parquet 파일 열기
table = pq.read_table(kor_path)
print(table)

ArrowInvalid: Could not open Parquet input source '/home/dudaji/Jun/llm-rag-chatbot/data/flores/flores_plus/devtest/kor_Hang.parquet': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [9]:
data_dir = "/home/dudaji/Jun/llm-rag-chatbot/data/flores/flores_plus/devtest"

eng_path = f"{data_dir}/eng_Latn.parquet"  # 영어 Parquet 파일 경로
kor_path = f"{data_dir}/kor_Hang.parquet"  # 한국어 Parquet 파일 경로

# Parquet 파일로부터 데이터 로드
dataset = load_parquet_data(eng_path, kor_path)

ArrowInvalid: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

번역 모듈 정의

In [4]:
# 번역 함수 (여기서는 간단히 예시 번역)
def translate(source_text, model=None):
    # 번역 모델을 호출하여 결과 생성
    # 이 예제에서는 단순히 소문자화된 텍스트를 반환 (모델이 있는 경우 교체 가능)
    return source_text.lower()


In [7]:
data_dir = "/home/dudaji/Jun/llm-rag-chatbot/data/flores/flores_plus/devtest"

eng_path = f"{data_dir}/eng_Latn.parquet"  # 영어 Parquet 파일 경로
kor_path = f"{data_dir}/kor_Hang.parquet"  # 한국어 Parquet 파일 경로

# Parquet 파일로부터 데이터 로드
dataset = load_parquet_data(eng_path, kor_path)

ArrowInvalid: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [ ]:
# 평가 수행


results = []
for _, row in dataset.iterrows():  # 각 샘플에 대해 평가 수행
    source = row["source"]
    reference = row["target"]

    # 번역 수행
    candidate = translate(source)

    # 평가 실행
    evaluation = evaluate(reference, candidate, "ko")
    results.append(
        {
            "Source": source,
            "Reference": reference,
            "Candidate": candidate,
            **evaluation,
        }
    )

# 결과 데이터프레임 생성
df = pd.DataFrame(results)
print(df)


In [ ]:

# 결과 저장 
df.to_csv("translation_evaluation_results.csv", index=False)
